# Load data and data analysis

In [ ]:
# TODO
# TODO: maybe load the dataset as follows from huggingface
# https://huggingface.co/docs/datasets/v1.9.0/loading_datasets.html
from datasets import load_dataset
dataset = load_dataset('squad')
# TODO: something similiar to this:
# TEMP
from datasets import load_dataset
test_set = load_dataset('squad', split='validation[:1000]')  # get first n entries from the test set
test_contexts = test_set['context']  # list of strings
test_questions = test_set['question']  # list of strings
test_answers = [d['text'] for d in test_set['answers']]  # list of lists of answers
#TEMP end

# Passage Ranking

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np

class PassageRanker:
  def __init__(self) -> None:
    self.model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

  def __get_most_similiar_sentence__(self, question, context):
    sentences = nltk.sent_tokenize(context)
    question_embedding = self.model.encode([question])  # array to get the correct format
    sentences_embedding = self.model.encode(sentences)
    cosine_similiarities = cosine_similarity(sentences_embedding, question_embedding)
    return sentences[np.argmax(cosine_similiarities)]
  
  def get_most_relevant_passages(self, questions, contexts):
    return [self.__get_most_similiar_sentence__(question, context) for (question, context) in zip(questions, contexts)]

In [ ]:
passage_ranker = PassageRanker()

# Setup generative model (open/closed model)

In [ ]:
from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
MAX_NUM_THREADS = 10
import torch
TORCH_DEVICE = torch.device("mps")

class GenerativeModel:
    def __init__(self, max_answer_length) -> None:
        self.generator = pipeline('text-generation', model="facebook/opt-1.3b")
        self.model = OPTForCausalLM.from_pretrained("facebook/opt-1.3b")
        self.model.to(TORCH_DEVICE)
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")
        self.tokenizer.padding_side = "left"   # so that the text will continue as without padding
        self.max_answer_length = max_answer_length

    def get_open_model_answer(self, question, context, use_pipeline=False):
        prompt = f"CONTEXT:\n{context}\nQUESTION:\n{question}"
        # generate answer
        answer = self._generate_answer(prompt, use_pipeline)
        # remove prompt from generated text
        answer = answer.removeprefix(prompt)
        return answer

    def get_closed_model_answer(self, question, use_pipeline=False):
        prompt = question
        # generate answer
        answer = self._generate_answer(prompt, use_pipeline)
        # remove prompt from generated text
        answer = answer.removeprefix(prompt)
        return answer

    def get_open_batch_answers(self, questions, contexts, use_pipeline=False):
        assert len(questions) == len(contexts), "questions and contexts should have the same length"
        prompts = [f"CONTEXT:\n{contexts[i]}\nQUESTION:\n{questions[i]}" for i in range(len(questions))]
        # generate answers
        answers = self._generate_batch_answers(prompts, use_pipeline)
        # remove prompts from generated text
        answers = [answers[i].removeprefix(prompts[i]) for i in range(len(prompts))]
        return answers

    # https://github.com/huggingface/transformers/issues/10704
    def get_closed_batch_answers(self, questions, use_pipeline=False):
        prompts = questions
        # generate answers
        answers = self._generate_batch_answers(prompts, use_pipeline)
        # remove prompts from generated text
        answers = [answers[i].removeprefix(prompts[i]) for i in range(len(prompts))]
        return answers
    
    def _generate_answer(self, prompt, use_pipeline):
        if use_pipeline:
            answer = self.generator(prompt, max_new_tokens=self.max_answer_length)[0]['generated_text']
        else:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(TORCH_DEVICE)
            generate_ids = self.model.generate(inputs.input_ids, max_new_tokens=self.max_answer_length)
            answer = self.tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        return answer

    def _generate_batch_answers(self, prompts, use_pipeline):
        if use_pipeline:
            answers = []
            with ThreadPoolExecutor(max_workers=MAX_NUM_THREADS) as executor:
                results = list(tqdm(executor.map(self.__generate_answer_multithread__, prompts), total=len(prompts)))
            answers.extend(results)
        else:
            inputs = self.tokenizer(prompts, return_tensors="pt", padding=True).to(TORCH_DEVICE)  # padding, so that all prompts have same length for computing it as a batch
            generate_ids = self.model.generate(inputs.input_ids, max_new_tokens=self.max_answer_length)
            answers = self.tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return answers
    
    def __generate_answer_multithread__(self, prompt):
        # Your answer generation logic here
        return self.generator(prompt, max_new_tokens=self.max_answer_length)[0]['generated_text']

In [ ]:
generative_model = GenerativeModel(max_answer_length=42)

In [ ]:
# test for single question
single_context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50."""
single_question = "Which NFL team represented the AFC at Super Bowl 50?"
single_correct_answers = ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']

print(f"Context: {single_context}\nQuestion: {single_question}\nCorrect answer: {single_correct_answers}")
print("Closed generative Model:")
print(generative_model.get_closed_model_answer(single_question))
print("Open generative Model:")
print(generative_model.get_open_model_answer(single_question, single_context))

# Evaluation

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import numpy as np
from transformers import pipeline
nltk.download('punkt')
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
MAX_NUM_THREADS = 10

In [ ]:
class ExtractiveModel:
  def __init__(self) -> None:
    model_name = "deepset/roberta-base-squad2"
    self.pipeline = pipeline('question-answering', model=model_name, tokenizer=model_name)

  def get_extractive_batch_answer(self, question, context):
    answer = self.pipeline(question=question, context=context)["answer"]
    return answer
  
  def get_extractive_batch_answers(self, questions, contexts):
    answers = []
    with ThreadPoolExecutor(max_workers=MAX_NUM_THREADS) as executor:
      results = list(tqdm(executor.map(self.get_extractive_batch_answer, questions, contexts), total=len(questions)))
    answers.extend(results)
    return answers

In [ ]:
extractive_model = ExtractiveModel()

In [ ]:
# test for single question
single_context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50."""
single_question = "Which NFL team represented the AFC at Super Bowl 50?"
single_correct_answers = ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']

print(f"Context: {single_context}\nQuestion: {single_question}\nCorrect answer: {single_correct_answers}")
print("Extractive Model:")
single_answer = extractive_model.get_extractive_batch_answer(single_question, single_context)
print(single_answer)

## Compute the answers from the different models

In [ ]:
# TEMP
from datasets import load_dataset
test_set = load_dataset('squad', split='validation[:1000]')  # get first n entries from the test set
test_contexts = test_set['context']  # list of strings
test_questions = test_set['question']  # list of strings
test_answers = [d['text'] for d in test_set['answers']]  # list of lists of answers
# passage ranking
test_contexts_passage_ranked = passage_ranker.get_most_relevant_passages(test_questions, test_contexts)
#TEMP end

In [ ]:
%%time
closed_answers = generative_model.get_closed_batch_answers(test_questions)

In [ ]:
%%time
open_answers = generative_model.get_open_batch_answers(test_questions, test_contexts_passage_ranked)

In [ ]:
%%time
extractive_answers = extractive_model.get_extractive_batch_answers(test_questions, test_contexts)

## Compute the scores for the answers

### Recall

In [ ]:
# NOTE: BLEU cannot be used, since the sentences are often very short, and BLEU uses 4-grams, so if the answer has less than 4 words, it gets a BLEU Score of 0.0

def compute_word_recall(correct_answer_, model_answer_):
  correct_answer_words = word_tokenize(correct_answer_)
  model_answer_words = set(word_tokenize(model_answer_))
  relevant_retrieved_words = np.sum([(corr_word in model_answer_words) for corr_word in correct_answer_words])
  all_relevant_words = len(correct_answer_words)
  return relevant_retrieved_words / all_relevant_words

def compute_word_recall_multiple_answers(correct_answers_single_question_, model_answer_):
  # return the best score, since the different answers are often synonyms and if one is taken, this is fine
  return np.max([compute_word_recall(correct_answer, model_answer_) for correct_answer in correct_answers_single_question_])

def compute_word_recall_average(correct_answers_, model_answers_):
  all_recalls = [compute_word_recall_multiple_answers(correct_answer, model_answer) for (correct_answer, model_answer) in zip(correct_answers_, model_answers_)]
  return np.mean(all_recalls)

In [ ]:
closed_recall = compute_word_recall_average(test_answers, closed_answers)
closed_recall

In [ ]:
open_recall = compute_word_recall_average(test_answers, open_answers)
open_recall

In [ ]:
extractive_recall = compute_word_recall_average(test_answers, extractive_answers)
extractive_recall

### Precision

In [ ]:
def compute_word_precision(correct_answer_, model_answer_):
  correct_answer_words = word_tokenize(correct_answer_)
  model_answer_words = set(word_tokenize(model_answer_))
  relevant_retrieved_words = np.sum([(corr_word in model_answer_words) for corr_word in correct_answer_words])
  all_retrieved_words = len(model_answer_words)
  if all_retrieved_words == 0:  # avoid division by zero
    precision = 0.0
  else:
    precision = relevant_retrieved_words / all_retrieved_words
  return precision

def compute_word_precision_multiple_answers(correct_answers_single_question_, model_answer_):
  # return the best score, since the different answers are often synonyms and if one is taken, this is fine
  return np.max([compute_word_precision(correct_answer, model_answer_) for correct_answer in correct_answers_single_question_])

def compute_word_precision_average(correct_answers_, model_answers_):
  all_recalls = [compute_word_precision_multiple_answers(correct_answer, model_answer) for (correct_answer, model_answer) in zip(correct_answers_, model_answers_)]
  return np.mean(all_recalls)

In [ ]:
closed_precision = compute_word_precision_average(test_answers, closed_answers)
closed_precision

In [ ]:
open_precision = compute_word_precision_average(test_answers, open_answers)
open_precision

In [ ]:
extractive_precision = compute_word_precision_average(test_answers, extractive_answers)
extractive_precision

## Plots